In [ ]:
import random
import pandas as pd
from operator import itemgetter
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoModelForSequenceClassification,AutoTokenizer,TrainingArguments
from trl import RewardTrainer

In [ ]:
df = pd.read_csv('feedback.csv')
df.head()

In [ ]:
df['tup'] = list(zip(df['answer'], df['feedback']))
df_g = df.groupby('question')['tup'].apply(list).reset_index()
df_g["sorted_tup"] = df_g["tup"].apply(lambda x :sorted(x,key=itemgetter(1)) )
df_g["chosen"] = df_g["sorted_tup"].apply(lambda x: x[-1][0])
df_g["chosen_score"] = df_g["sorted_tup"].apply(lambda x: x[-1][1])
df_g["rejected"] = df_g["sorted_tup"].apply(lambda x: x[0][0])
df_g["rejected_score"] = df_g["sorted_tup"].apply(lambda x: x[0][1])
df_g = df_g.dropna()
df_g = df_g[(df_g['chosen_score']>=4.0) & (df_g['rejected_score']<4.0)]
df_g.to_csv("feedback_comparison_dataset.csv")

In [ ]:
rows = []
for record in df_g.itertuples(index=True, name='Pandas'):
    if record is None or len(record) == 0:
        continue
    rows.append({
        "instruction": record.question,
        "chosen_response": record.chosen,
        "rejected_response": record.rejected
    })

prepared_dataset = Dataset.from_list(rows)
prepared_dataset.to_pandas()

In [ ]:
#Select a base model whch we need to train for reward modeling.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_name = "distilroberta-base"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1,)  # device_map="auto",  offload_state_dict=False
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.to(device)

In [ ]:
print(model.hf_device_map)

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}

    prompt_plus_chosen_response = examples["instruction"] + "\n" + examples["chosen_response"]
    prompt_plus_rejected_response = examples["instruction"] + "\n" + examples["rejected_response"]
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }

formatted_dataset = prepared_dataset.map(formatting_func)
formatted_dataset = formatted_dataset.train_test_split()

In [8]:
training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs = 10,
    report_to=None,

)

trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["test"],

)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [9]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/home/sidmadala/miniconda3/envs/cs443/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/sidmadala/miniconda3/envs/cs443/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/sidmadala/miniconda3/envs/cs443/lib/python3.10/site-packages/transformers/models/roberta/modeling_roberta.py", line 1191, in forward
    outputs = self.roberta(
  File "/home/sidmadala/miniconda3/envs/cs443/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/sidmadala/miniconda3/envs/cs443/lib/python3.10/site-packages/transformers/models/roberta/modeling_roberta.py", line 828, in forward
    encoder_outputs = self.encoder(
  File "/home/sidmadala/mi

In [ ]:
trainer.save_model("./models/test_reward_model") 

In [1]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
config = GPT2Config.from_pretrained('gpt2')  # Load the base GPT-2 configuration
model = GPT2LMHeadModel(config)
model.load_state_dict(torch.load('rlhf_expts_ckpts/best_model.pth'))
model.save_pretrained('rlhf_expts_ckpts/gpt2_ppo')

/home/sidmadala/miniconda3/envs/cs443/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'torch' is not defined